# creating voting affinity scores for members of the 118th congress

In [16]:
import pandas as pd
import numpy as np

In [17]:
votes = pd.read_csv("https://voteview.com/static/data/out/votes/HS118_votes.csv")

In [18]:
votes

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.0
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
251429,118,Senate,253,42304,6,100.0
251430,118,Senate,253,42305,1,NaN
251431,118,Senate,253,49308,1,100.0
251432,118,Senate,253,49703,1,99.0


In [19]:
votes = votes[['chamber',	'rollnumber',	'icpsr',	'cast_code'	]]
senate_votes = votes.query("chamber=='Senate'")

In [42]:
senate_votes

,chamber,rollnumber,icpsr,cast_code
226138,Senate,1,14226,1
226139,Senate,1,14435,1
226140,Senate,1,14858,1
226141,Senate,1,14871,1
226142,Senate,1,14921,1
...,...,...,...,...
251429,Senate,253,42304,6
251430,Senate,253,42305,1
251431,Senate,253,49308,1
251432,Senate,253,49703,1


In [44]:
paired_votes = pd.merge(senate_votes,senate_votes, on = ["chamber",	"rollnumber"],how = "inner")

In [45]:
paired_votes

,chamber,rollnumber,icpsr_x,cast_code_x,icpsr_y,cast_code_y
0,Senate,1,14226,1,14226,1
1,Senate,1,14226,1,14435,1
2,Senate,1,14226,1,14858,1
3,Senate,1,14226,1,14871,1
4,Senate,1,14226,1,14921,1
...,...,...,...,...,...,...
2529199,Senate,253,91300,1,42304,6
2529200,Senate,253,91300,1,42305,1
2529201,Senate,253,91300,1,49308,1
2529202,Senate,253,91300,1,49703,1


In [48]:
paired_votes = paired_votes.rename({"icpsr_x":"icpsr",
                                   "icpsr_y":"member_compare",
                                   "cast_code_x":"vote",
                                   "cast_code_y":"compare_vote"},axis=1)



In [49]:
paired_votes

,chamber,rollnumber,icpsr,vote,member_compare,compare_vote
0,Senate,1,14226,1,14226,1
1,Senate,1,14226,1,14435,1
2,Senate,1,14226,1,14858,1
3,Senate,1,14226,1,14871,1
4,Senate,1,14226,1,14921,1
...,...,...,...,...,...,...
2529199,Senate,253,91300,1,42304,6
2529200,Senate,253,91300,1,42305,1
2529201,Senate,253,91300,1,49308,1
2529202,Senate,253,91300,1,49703,1


In [56]:
paired_votes = paired_votes.query("icpsr != member_compare")
paired_votes

,chamber,rollnumber,icpsr,vote,member_compare,compare_vote,equal_votes
1,Senate,1,14226,1,14435,1,True
2,Senate,1,14226,1,14858,1,True
3,Senate,1,14226,1,14871,1,True
4,Senate,1,14226,1,14921,1,True
5,Senate,1,14226,1,15015,1,True
...,...,...,...,...,...,...,...
2529198,Senate,253,91300,1,42303,6,False
2529199,Senate,253,91300,1,42304,6,False
2529200,Senate,253,91300,1,42305,1,True
2529201,Senate,253,91300,1,49308,1,True


In [57]:
# paired_votes['equal_vote'] = paired_votes['vote'] == 
# paired_votes["compared_vote"]

paired_votes = paired_votes.assign(equal_votes = paired_votes["vote"]==paired_votes["compare_vote"])

In [58]:
paired_votes

,chamber,rollnumber,icpsr,vote,member_compare,compare_vote,equal_votes
1,Senate,1,14226,1,14435,1,True
2,Senate,1,14226,1,14858,1,True
3,Senate,1,14226,1,14871,1,True
4,Senate,1,14226,1,14921,1,True
5,Senate,1,14226,1,15015,1,True
...,...,...,...,...,...,...,...
2529198,Senate,253,91300,1,42303,6,False
2529199,Senate,253,91300,1,42304,6,False
2529200,Senate,253,91300,1,42305,1,True
2529201,Senate,253,91300,1,49308,1,True


In [59]:
affinity = paired_votes.groupby(['icpsr','member_compare'])\
.agg({'equal_votes':"mean"}).reset_index()
affinity

,icpsr,member_compare,equal_votes
0,14226,14435,0.308300
1,14226,14858,0.379447
2,14226,14871,0.355731
3,14226,14921,0.766798
4,14226,15015,0.359684
...,...,...,...
10093,91300,42304,0.177866
10094,91300,42305,1.000000
10095,91300,49300,0.485597
10096,91300,49308,0.913043


In [65]:
affinity["affinity_score"] = round(100*affinity["equal_votes"],1).astype("str")+"%"

In [66]:
affinity

,icpsr,member_compare,equal_votes,affinity_score
0,14226,14435,0.308300,30.8%
1,14226,14858,0.379447,37.9%
2,14226,14871,0.355731,35.6%
3,14226,14921,0.766798,76.7%
4,14226,15015,0.359684,36.0%
...,...,...,...,...
10093,91300,42304,0.177866,17.8%
10094,91300,42305,1.000000,100.0%
10095,91300,49300,0.485597,48.6%
10096,91300,49308,0.913043,91.3%


In [67]:
affinity = affinity.drop('equal_votes',axis = 1)
affinity

,icpsr,member_compare,affinity_score
0,14226,14435,30.8%
1,14226,14858,37.9%
2,14226,14871,35.6%
3,14226,14921,76.7%
4,14226,15015,36.0%
...,...,...,...
10093,91300,42304,17.8%
10094,91300,42305,100.0%
10095,91300,49300,48.6%
10096,91300,49308,91.3%


In [68]:
ideology = pd.read_csv("https://voteview.com/static/data/out/members/HS118_members.csv")

In [69]:
ideology

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,118,House,20301,41,3,AL,200,NaN,NaN,"ROGERS, Mike Dennis",...,NaN,0.374,0.397,-72.70003,0.85526,465.0,37.0,NaN,0.539,0.341
1,118,House,21102,41,7,AL,100,NaN,NaN,"SEWELL, Terri",...,NaN,-0.396,0.399,-21.42846,0.95257,441.0,10.0,NaN,-0.534,0.445
2,118,House,21500,41,6,AL,200,NaN,NaN,"PALMER, Gary James",...,NaN,0.678,0.102,-41.77066,0.91320,460.0,25.0,NaN,0.662,0.113
3,118,House,22108,41,1,AL,200,NaN,NaN,"CARL, Jerry L.",...,NaN,0.519,0.313,-53.08103,0.89256,467.0,23.0,NaN,0.504,0.133
4,118,House,22140,41,2,AL,200,NaN,NaN,"MOORE, Barry",...,NaN,0.638,-0.158,-38.27149,0.92115,466.0,12.0,NaN,0.740,-0.148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,118,Senate,40915,56,0,WV,100,NaN,NaN,"MANCHIN, Joe, III",...,NaN,-0.058,0.404,-40.16046,0.84352,236.0,10.0,NaN,-0.020,0.290
538,118,Senate,29940,25,0,WI,100,NaN,NaN,"BALDWIN, Tammy",...,NaN,-0.492,-0.135,-10.32065,0.95791,240.0,5.0,NaN,-0.428,-0.206
539,118,Senate,41111,25,0,WI,200,NaN,NaN,"JOHNSON, Ron",...,NaN,0.631,-0.145,-26.53011,0.89410,237.0,9.0,NaN,0.694,0.111
540,118,Senate,20953,68,0,WY,200,NaN,NaN,"LUMMIS, Cynthia M.",...,NaN,0.694,-0.339,-29.20122,0.88315,235.0,15.0,NaN,0.745,-0.063


In [71]:
ideo_senate = ideology.query("chamber == 'Senate'")
ideo_senate = ideo_senate[['icpsr', 'state_abbrev','party_code','bioname']]
ideo_senate

,icpsr,state_abbrev,party_code,bioname
441,42102,AL,200,"TUBERVILLE, Thomas Hawley (Tommy)"
442,42300,AL,200,"BRITT, Katie Elizabeth"
443,40300,AK,200,"MURKOWSKI, Lisa"
444,41500,AK,200,"SULLIVAN, Daniel Scott"
445,41905,AZ,100,"KELLY, Mark Edward"
...,...,...,...,...
537,40915,WV,100,"MANCHIN, Joe, III"
538,29940,WI,100,"BALDWIN, Tammy"
539,41111,WI,200,"JOHNSON, Ron"
540,20953,WY,200,"LUMMIS, Cynthia M."


In [72]:
affinity = pd.merge(affinity,ideo_senate,
                   on = "icpsr", 
                   how = "outer",validate ="many_to_one")
affinity


,icpsr,member_compare,affinity_score,state_abbrev,party_code,bioname
0,14226,14435,30.8%,IA,200,"GRASSLEY, Charles Ernest"
1,14226,14858,37.9%,IA,200,"GRASSLEY, Charles Ernest"
2,14226,14871,35.6%,IA,200,"GRASSLEY, Charles Ernest"
3,14226,14921,76.7%,IA,200,"GRASSLEY, Charles Ernest"
4,14226,15015,36.0%,IA,200,"GRASSLEY, Charles Ernest"
...,...,...,...,...,...,...
10093,91300,42304,17.8%,AZ,328,"SINEMA, Kyrsten"
10094,91300,42305,100.0%,AZ,328,"SINEMA, Kyrsten"
10095,91300,49300,48.6%,AZ,328,"SINEMA, Kyrsten"
10096,91300,49308,91.3%,AZ,328,"SINEMA, Kyrsten"


In [74]:
affinity = pd.merge(affinity,ideo_senate,
                   left_on = "member_compare", 
                    right_on = "icpsr",
                   how = "outer",validate ="many_to_one")
affinity

,icpsr_x,member_compare,affinity_score,state_abbrev_x,party_code_x,bioname_x,icpsr_y,state_abbrev_y,party_code_y,bioname_y
0,14226,14435,30.8%,IA,200,"GRASSLEY, Charles Ernest",14435,MA,100,"MARKEY, Edward John"
1,14858,14435,90.9%,NY,100,"SCHUMER, Charles Ellis (Chuck)",14435,MA,100,"MARKEY, Edward John"
2,14871,14435,93.3%,OR,100,"WYDEN, Ronald Lee",14435,MA,100,"MARKEY, Edward John"
3,14921,14435,24.1%,KY,200,"McCONNELL, Addison Mitchell (Mitch)",14435,MA,100,"MARKEY, Edward John"
4,15015,14435,92.1%,DE,100,"CARPER, Thomas Richard",14435,MA,100,"MARKEY, Edward John"
...,...,...,...,...,...,...,...,...,...,...
10093,42305,14226,33.3%,CA,100,"BUTLER, Laphonza Romanique",14226,IA,200,"GRASSLEY, Charles Ernest"
10094,49300,14226,14.0%,CA,100,"FEINSTEIN, Dianne",14226,IA,200,"GRASSLEY, Charles Ernest"
10095,49308,14226,36.4%,WA,100,"MURRAY, Patty",14226,IA,200,"GRASSLEY, Charles Ernest"
10096,49703,14226,60.1%,ME,200,"COLLINS, Susan Margaret",14226,IA,200,"GRASSLEY, Charles Ernest"


In [76]:
affinity.query("state_abbrev_x = 'VA'" ).sort_values(by="affinity_score")

ValueError: cannot assign without a target object